In [2]:
import pandas as pd
import numpy as np
from pprint import pprint

In [3]:
summary_df = pd.read_csv("summary_df.csv", index_col=0, parse_dates=True)
ranking_df = pd.read_csv("ranking_df.csv", index_col=0, parse_dates=True)
views_df = pd.read_csv("views_df.csv", index_col=0, parse_dates=True)
earnings_df = pd.read_csv("earnings_df.csv", index_col=0, parse_dates=True)

In [4]:
combined_df = pd.merge(summary_df, ranking_df, on='name')
combined_df = pd.merge(combined_df, views_df, on='name')
combined_df = pd.merge(combined_df, earnings_df, on='name')

In [5]:
combined_df.count()

category_x              1315
created                 1626
name                    1626
subscribers             1626
uploads                 1626
views                   1626
category_y              1315
grade                   1626
socialblade_rank        1626
subscriber_rank         1626
view_rank               1626
category_x              1315
subs_last30d_count      1626
views_last30d_count     1626
category_y              1315
max_annual_earnings     1626
max_monthly_earnings    1626
min_annual_earnings     1626
min_monthly_earnings    1626
dtype: int64

In [6]:
cols=pd.Series(combined_df.columns)
for dup in combined_df.columns.get_duplicates(): cols[combined_df.columns.get_loc(dup)]=[dup+'.'+str(d_idx) if d_idx!=0 else dup for d_idx in range(combined_df.columns.get_loc(dup).sum())]
combined_df.columns=cols
combined_df = combined_df[["category_x.1", "created", "name", "subscribers", "uploads", "views", "grade", "socialblade_rank", "subscriber_rank", "view_rank", "subs_last30d_count", "views_last30d_count","min_monthly_earnings", "max_monthly_earnings", "min_annual_earnings", "max_annual_earnings"]]
combined_df = combined_df.dropna().rename(columns={"category_x.1":"category"})
combined_df = combined_df.rename(columns={"min_monthly_earnings":"monthly_earnings"})

C:\Users\Andrew Chin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: 'get_duplicates' is deprecated and will be removed in a future release. You can use idx[idx.duplicated()].unique() instead
  


In [7]:
combined_df = combined_df[combined_df['subscribers'] >= 0]
combined_df = combined_df[combined_df['views'] > 0]
combined_df = combined_df[combined_df['subs_last30d_count'] > 0]
combined_df = combined_df[combined_df['views_last30d_count'] > 0]
combined_df = combined_df[combined_df['monthly_earnings'] > 0]

In [8]:
combined_df.count()

category                1121
created                 1121
name                    1121
subscribers             1121
uploads                 1121
views                   1121
grade                   1121
socialblade_rank        1121
subscriber_rank         1121
view_rank               1121
subs_last30d_count      1121
views_last30d_count     1121
monthly_earnings        1121
max_monthly_earnings    1121
min_annual_earnings     1121
max_annual_earnings     1121
dtype: int64

In [9]:
category_avgmonthlyearnings = combined_df.groupby(['category'])['monthly_earnings'].mean().astype(np.int64)
category_avgmonthlyearnings = pd.DataFrame(category_avgmonthlyearnings)
category_avgmonthlyearnings

,monthly_earnings
category,
Animals,11120
Autos,6455
Comedy,13639
Education,20026
Entertainment,13509
Film,11943
Games,11325
Howto,15150
Music,15473


In [10]:
category_avgsubscribers = combined_df.groupby(['category'])['subscribers'].mean().astype(np.int64)
category_avgsubscribers = pd.DataFrame(category_avgsubscribers)

In [11]:
category_avguploads = combined_df.groupby(['category'])['uploads'].mean().astype(np.int64)
category_avguploads = pd.DataFrame(category_avguploads)

In [12]:
category_avgviews = combined_df.groupby(['category'])['views'].mean().astype(np.int64)
category_avgviews = pd.DataFrame(category_avgviews)

In [13]:
earnings_subscribers = pd.merge(category_avgmonthlyearnings, category_avgsubscribers, on='category')
earnings_subscribers

,monthly_earnings,subscribers
category,,
Animals,11120,2141071
Autos,6455,1358424
Comedy,13639,5008017
Education,20026,3377660
Entertainment,13509,2975862
Film,11943,2200461
Games,11325,4437635
Howto,15150,5729361
Music,15473,3993768


In [14]:
earnings_uploads = pd.merge(category_avgmonthlyearnings, category_avguploads, on='category')
earnings_uploads

,monthly_earnings,uploads
category,,
Animals,11120,696
Autos,6455,661
Comedy,13639,1199
Education,20026,488
Entertainment,13509,4776
Film,11943,908
Games,11325,1995
Howto,15150,731
Music,15473,1763


In [15]:
earnings_views = pd.merge(category_avgmonthlyearnings, category_avgviews, on='category')
earnings_views

,monthly_earnings,views
category,,
Animals,11120,738694619
Autos,6455,383536914
Comedy,13639,1297376517
Education,20026,1589255902
Entertainment,13509,1279749914
Film,11943,1035872918
Games,11325,1746559285
Howto,15150,1225525979
Music,15473,1983268280


In [16]:
earnings_subscribers.to_csv('earnings_subscribers.csv')
earnings_uploads.to_csv('earnings_uploads.csv')
earnings_views.to_csv('earnings_views.csv')

In [20]:
timeline_df = pd.read_csv("timeline_df.csv")
timeline_df = timeline_df[["date", "category", "min_earnings", "subscribers", "views"]].dropna().rename(columns={"min_earnings":"earnings"})
timeline_df['date'] = pd.to_datetime(timeline_df['date'])
timeline_df['subscribers'] = timeline_df.subscribers.str.replace(' LIVE', '')

start_date = '2018-12-03'
end_date = '2018-12-16'
mask = (timeline_df['date'] >= start_date) & (timeline_df['date'] <= end_date)
timeline_df = timeline_df.loc[mask]

timeline_df['earnings'] = timeline_df['earnings'].convert_objects(convert_numeric=True)
timeline_df['subscribers'] = timeline_df['subscribers'].convert_objects(convert_numeric=True)
timeline_df['views'] = timeline_df['views'].convert_objects(convert_numeric=True)

timeline_df = timeline_df[timeline_df['earnings'] > 0]
timeline_df = timeline_df[timeline_df['subscribers'] > 0]
timeline_df = timeline_df[timeline_df['views'] > 0]

timeline_df.to_csv('timeline_df_refined.csv')

C:\Users\Andrew Chin\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Andrew Chin\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if sys.path[0] == '':
C:\Users\Andrew Chin\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_num

In [21]:
timeline_df = pd.read_csv("timeline_df_refined.csv", index_col=0, parse_dates=True)
timeline_df = timeline_df.groupby(['category', 'date']).mean().astype(np.int64)
timeline_df.to_csv('timeline_averages.csv')

In [22]:
timeline_averages = pd.read_csv("timeline_averages.csv", index_col=0, parse_dates=True)
timeline_averages

,date,earnings,subscribers,views
category,,,,
Animals,2018-12-03,10640,2083417,735286614
Animals,2018-12-04,6316,1902359,669109801
Animals,2018-12-05,6020,2095582,735624820
Animals,2018-12-06,6000,2102038,725683869
Animals,2018-12-07,5818,2107472,735924726
Animals,2018-12-08,6116,1923684,673714589
Animals,2018-12-09,5500,1928049,679565975
Animals,2018-12-10,5360,2122846,731133352
Animals,2018-12-11,5780,2127169,736440628
